<a href="https://colab.research.google.com/github/ElmonChan/ICU_Mortality_Prediction/blob/main/ICUSTAYID_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import datetime 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
admission = pd.read_parquet('/content/drive/MyDrive/BMI707_Final/707/processed/ADMISSIONS.parquet')
icu = pd.read_parquet('/content/drive/MyDrive/BMI707_Final/707/processed/ICUSTAYS.parquet')

In [ ]:
admission.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,None,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,None,None,UNSTABLE ANGINA\CATH,0,1
11,32,33,176176,2116-12-23 22:30:00,2116-12-27 12:05:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,None,PROTESTANT QUAKER,MARRIED,UNKNOWN/NOT SPECIFIED,2116-12-23 14:42:00,2116-12-23 23:41:00,SEPSIS;TELEMETRY,0,1
22,456,357,174486,2197-12-06 07:13:00,2198-01-03 14:00:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2197-12-06 06:28:00,2197-12-06 07:43:00,GASTROINTESTINAL BLEED,0,1
23,457,357,145674,2198-08-02 04:49:00,2198-10-26 14:55:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-08-02 02:03:00,2198-08-02 05:37:00,SEIZURE,0,1
24,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-11-01 18:01:00,2198-11-01 23:06:00,SEPSIS,0,1


In [ ]:
icu.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
10,375,279,192224,204407,carevue,MICU,MICU,15,15,2164-06-14 21:18:27,2164-06-18 22:53:12,4.0658
14,379,283,109185,231490,carevue,MICU,MICU,15,15,2166-08-12 22:03:26,2166-09-12 14:41:42,30.6932
15,380,283,144156,280531,carevue,SICU,SICU,23,23,2166-10-02 15:38:43,2166-10-07 13:22:00,4.9051
27,392,292,179726,222505,carevue,MICU,MICU,12,12,2103-09-27 18:29:30,2103-09-28 15:44:31,0.8854


## Extract ICU with outime near death time

In [ ]:
dead_patients = admission.loc[admission.DISCHARGE_LOCATION=='DEAD/EXPIRED',['SUBJECT_ID','HADM_ID','ADMITTIME','DEATHTIME']]
dead_patients.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DEATHTIME
130,124,138376,2165-12-31 18:55:00,2166-02-01 06:55:00
193,109,189332,2142-08-28 19:48:00,2142-08-30 15:20:00
346,491,157083,2170-04-02 21:41:00,2170-04-05 06:24:00
351,495,186199,2183-05-13 15:08:00,2183-05-22 09:32:00
374,377,139824,2168-03-04 13:56:00,2168-04-04 10:47:00


In [ ]:
dead_icu = dead_patients.set_index('HADM_ID').join(icu.set_index('HADM_ID'),lsuffix='_dead',rsuffix='_icu')
dead_icu.head()

,SUBJECT_ID_dead,ADMITTIME,DEATHTIME,ROW_ID,SUBJECT_ID_icu,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
HADM_ID,,,,,,,,,,,,,,
100061,11728,2178-12-25 14:11:00,2178-12-27 11:45:00,14932.0,11728,201906.0,metavision,CCU,CCU,7.0,7.0,2178-12-25 14:12:10,2178-12-27 16:30:03,2.0958
100087,42255,2126-11-01 17:47:00,2126-11-06 05:00:00,42085.0,42255,294404.0,metavision,CCU,CCU,7.0,7.0,2126-11-01 17:47:40,2126-11-05 11:17:03,3.7287
100098,65411,2108-04-04 08:00:00,2108-05-08 07:07:00,49961.0,65411,216749.0,metavision,TSICU,TSICU,14.0,14.0,2108-04-04 15:26:39,2108-05-08 14:47:19,33.9727
100124,19644,2103-07-24 18:11:00,2103-07-26 16:47:00,24958.0,19644,218755.0,carevue,MICU,MICU,15.0,15.0,2103-07-24 18:14:00,2103-07-26 16:47:00,1.9396
100197,90629,2136-10-23 16:23:00,2136-10-24 12:55:00,58349.0,90629,203563.0,metavision,SICU,SICU,57.0,57.0,2136-10-23 16:23:52,2136-10-24 15:59:24,0.9830


In [ ]:
dead_icu = dead_icu.assign(DEATHTIME=pd.to_datetime(dead_icu.DEATHTIME),
                           OUTTIME=pd.to_datetime(dead_icu.OUTTIME))
dead_icu = dead_icu.loc[dead_icu['OUTTIME'].between(dead_icu.DEATHTIME - pd.Timedelta(1,'d'), dead_icu.DEATHTIME + pd.Timedelta(1,'d')),:]

In [ ]:
dead_icu

,SUBJECT_ID_dead,ADMITTIME,DEATHTIME,ROW_ID,SUBJECT_ID_icu,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
HADM_ID,,,,,,,,,,,,,,
100061,11728,2178-12-25 14:11:00,2178-12-27 11:45:00,14932.0,11728,201906.0,metavision,CCU,CCU,7.0,7.0,2178-12-25 14:12:10,2178-12-27 16:30:03,2.0958
100087,42255,2126-11-01 17:47:00,2126-11-06 05:00:00,42085.0,42255,294404.0,metavision,CCU,CCU,7.0,7.0,2126-11-01 17:47:40,2126-11-05 11:17:03,3.7287
100098,65411,2108-04-04 08:00:00,2108-05-08 07:07:00,49961.0,65411,216749.0,metavision,TSICU,TSICU,14.0,14.0,2108-04-04 15:26:39,2108-05-08 14:47:19,33.9727
100124,19644,2103-07-24 18:11:00,2103-07-26 16:47:00,24958.0,19644,218755.0,carevue,MICU,MICU,15.0,15.0,2103-07-24 18:14:00,2103-07-26 16:47:00,1.9396
100197,90629,2136-10-23 16:23:00,2136-10-24 12:55:00,58349.0,90629,203563.0,metavision,SICU,SICU,57.0,57.0,2136-10-23 16:23:52,2136-10-24 15:59:24,0.9830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199605,70516,2189-02-12 23:22:00,2189-03-27 21:00:00,51734.0,70516,256267.0,metavision,MICU,MICU,50.0,50.0,2189-02-12 23:22:52,2189-03-28 03:17:10,43.1627
199622,5476,2139-12-05 14:55:00,2139-12-05 21:20:00,6955.0,5476,216125.0,metavision,CCU,CCU,7.0,7.0,2139-12-05 14:56:25,2139-12-05 23:32:28,0.3584
199636,17457,2196-10-31 13:00:00,2196-11-03 07:00:00,22154.0,17457,246530.0,carevue,CSRU,CSRU,14.0,14.0,2196-10-31 15:57:10,2196-11-03 07:00:11,2.6271


In [ ]:
dead_icu.SUBJECT_ID_dead.value_counts()

11728    1
81849    1
22499    1
13599    1
82148    1
        ..
743      1
60659    1
95182    1
85979    1
64296    1
Name: SUBJECT_ID_dead, Length: 1164, dtype: int64

In [ ]:
icu['label'] = icu.ICUSTAY_ID.isin(dead_icu.ICUSTAY_ID).astype(int)

In [ ]:
icu

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,label
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600,0
10,375,279,192224,204407,carevue,MICU,MICU,15,15,2164-06-14 21:18:27,2164-06-18 22:53:12,4.0658,0
14,379,283,109185,231490,carevue,MICU,MICU,15,15,2166-08-12 22:03:26,2166-09-12 14:41:42,30.6932,0
15,380,283,144156,280531,carevue,SICU,SICU,23,23,2166-10-02 15:38:43,2166-10-07 13:22:00,4.9051,0
27,392,292,179726,222505,carevue,MICU,MICU,12,12,2103-09-27 18:29:30,2103-09-28 15:44:31,0.8854,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61518,59797,94924,186391,290051,metavision,SICU,SICU,57,57,2134-01-08 21:59:39,2134-01-09 18:29:16,0.8539,0
61519,59798,94924,151078,225864,metavision,SICU,SICU,33,33,2134-03-11 14:04:54,2134-03-12 12:14:42,0.9235,0
61523,59802,94937,162005,213159,metavision,CSRU,SICU,12,33,2106-12-25 11:42:23,2107-01-10 11:23:18,15.9867,0
61524,59803,94937,162005,280084,metavision,CSRU,CSRU,15,15,2107-01-18 11:11:21,2107-01-19 14:37:37,1.1432,0


## Write file

In [ ]:
!head /content/drive/MyDrive/BMI707_Final/707/data/ICUSTAY_ID_positive.csv

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,OUTTIME
27,292,179726,222505,2103-09-27 18:29:30,2103-09-28 15:44:31
196,109,189332,287547,2142-08-28 19:49:27,2142-08-30 16:10:05
216,124,138376,256064,2166-01-09 13:05:18,2166-02-01 10:52:09
302,188,132401,292343,2162-01-10 16:36:40,2162-01-16 18:35:07
584,871,141369,246188,2125-05-04 03:16:00,2125-05-11 14:44:00
637,906,113399,210602,2171-04-02 22:50:00,2171-04-16 07:49:00
640,907,133125,207646,2163-10-01 22:51:53,2163-10-02 17:33:37
776,1331,162098,297131,2130-02-11 02:11:20,2130-02-13 11:17:54
888,377,139824,267694,2168-03-26 08:03:00,2168-04-03 15:03:00


In [ ]:
icu.loc[:,["SUBJECT_ID","HADM_ID","ICUSTAY_ID","INTIME","OUTTIME","label"]].to_csv('/content/drive/MyDrive/BMI707_Final/707/data/ICU_label.csv',index=False)

In [ ]:
dead_icu.ICUSTAY_ID.astype('int').to_csv('/content/drive/MyDrive/BMI707_Final/data/ICUSTAY_ID_positive.csv',index=False,header=False)

In [ ]:
notDead_id = pd.Series(list(set(icu.ICUSTAY_ID) - set(dead_icu.ICUSTAY_ID)))
notDead_id.to_csv('/content/drive/MyDrive/BMI707_Final/data/ICUSTAY_ID_negative.csv',index=False,header=False)

In [ ]:
# Check icu stay range
test = pd.DataFrame({'intime':pd.to_datetime(dead_icu.INTIME),
                     'outtime':pd.to_datetime(dead_icu.OUTTIME),
                     'ICUSTAY_ID':dead_icu.ICUSTAY_ID})x
test['diff'] = test.outtime-test.intime

In [ ]:
test['diff'].quantile([0.1,.2,.3,.4,.5,.6,.7,.75,.8,.9,1])

0.10      0 days 14:13:15.900000
0.20             1 days 03:15:26
0.30      1 days 19:47:50.300000
0.40      2 days 13:40:07.200000
0.50             3 days 18:49:07
0.60      5 days 08:44:49.400000
0.70   7 days 15:12:32.799999999
0.75      9 days 00:05:51.500000
0.80     11 days 05:38:41.200000
0.90     17 days 08:29:06.100000
1.00            65 days 14:18:46
Name: diff, dtype: timedelta64[ns]

In [ ]:
test.diff

<bound method DataFrame.diff of                      intime             outtime             diff
HADM_ID                                                         
100061  2178-12-25 14:12:10 2178-12-27 16:30:03  2 days 02:17:53
100087  2126-11-01 17:47:40 2126-11-05 11:17:03  3 days 17:29:23
100098  2108-04-04 15:26:39 2108-05-08 14:47:19 33 days 23:20:40
100124  2103-07-24 18:14:00 2103-07-26 16:47:00  1 days 22:33:00
100197  2136-10-23 16:23:52 2136-10-24 15:59:24  0 days 23:35:32
...                     ...                 ...              ...
199605  2189-02-12 23:22:52 2189-03-28 03:17:10 43 days 03:54:18
199622  2139-12-05 14:56:25 2139-12-05 23:32:28  0 days 08:36:03
199636  2196-10-31 15:57:10 2196-11-03 07:00:11  2 days 15:03:01
199811  2125-06-20 21:21:48 2125-06-25 16:21:06  4 days 18:59:18
199949  2181-05-15 18:20:51 2181-05-20 22:45:51  5 days 04:25:00

[1164 rows x 3 columns]>

In [ ]:
test['diff'].sort_values().tail(100)

HADM_ID
163271   18 days 19:58:27
120296   18 days 20:47:52
181911   18 days 20:59:47
115146   18 days 23:36:36
102870   19 days 01:28:00
               ...       
181414   51 days 19:26:05
133854   53 days 06:03:01
116969   53 days 10:26:13
194762   62 days 17:39:27
177998   65 days 14:18:46
Name: diff, Length: 100, dtype: timedelta64[ns]